In [1]:
import os 
import sys 
import numpy as np

sys.path.append(r'../')

from methods import LMEO, Static
from pprint import pprint 

In [2]:
lm = LMEO()
london_words = Static.get_words_dict()['London']
birmingham_words = Static.get_words_dict()['Birmingham']
all_words = Static.get_words_dict()

### We will do the telescope experiment

1) Order the sets.

In [23]:
city = 'Birmingham'

hotels_3_prompt = f"Recommend three hotels in {city} UK. \n State the name of the hotel on a new line each time not using a numbered list."
hotels_5_prompt = f"Recommend five hotels in {city} UK. \n State the name of the hotel on a new line each time not using a numbered list."
hotel_words = all_words[city]

In [24]:
print(hotel_words)

['ACHotel', 'Hyatt', 'Metro', 'Marriot', 'adisson', 'almaison', 'BirminghamCityCentre', 'Vin', 'Cube', 'dgbaston', 'Grand', 'Staying', 'Indigo', 'Hampton']


In [25]:
top3_ranks = lm.order_set(hotels_3_prompt, f'{city}Top3_1000', 1000, num_ranks=3)
top5_ranks = lm.order_set(hotels_5_prompt, f'{city}Top5_1000', 1000, num_ranks=5)

top3_ranks_stacked = np.column_stack(top3_ranks)
top5_ranks_stacked = np.column_stack(top5_ranks)

Loading Previously Generated Ranks.
Loading Previously Generated Ranks.


In [26]:
assert len(top3_ranks_stacked) == 3
assert len(top5_ranks_stacked) == 5

In [27]:
top3_percent_dict = LMEO.create_percent_dict(top3_ranks_stacked, hotel_words, 1)
top5_percent_dict = LMEO.create_percent_dict(top5_ranks_stacked, hotel_words, 1)

In [28]:
pprint(top3_percent_dict)

{0: {'ACHotel': 0.0031217481789802292,
     'Grand': 0.007284079084287203,
     'Hampton': 0.001040582726326743,
     'Hyatt': 0.7700312174817983,
     'Indigo': 0.001040582726326743,
     'Marriot': 0.005202913631633716,
     'Metro': 0.020811654526534867,
     'Other': 0.006243496357960459,
     'Staying': 0.013527575442247664,
     'Vin': 0.026014568158168584,
     'adisson': 0.006243496357960459,
     'almaison': 0.08949011446409992,
     'dgbaston': 0.049947970863683626},
 1: {'ACHotel': 0.008324661810613946,
     'BirminghamCityCentre': 0.004162330905306972,
     'Cube': 0.001040582726326743,
     'Grand': 0.004162330905306972,
     'Hampton': 0.01873048907388138,
     'Hyatt': 0.046826222684703406,
     'Indigo': 0.008324661810613946,
     'Marriot': 0.006243496357960459,
     'Metro': 0.005202913631633716,
     'Other': 0.039542143600416226,
     'Staying': 0.035379812695109265,
     'Vin': 0.1425598335067638,
     'adisson': 0.011446409989594177,
     'almaison': 0.60874089490

In [29]:
def longest_run(list1, list2):
    # Create a table to store the lengths of common subsequences
    n = len(list1)
    m = len(list2)
    table = [[0] * (m + 1) for _ in range(n + 1)]
    longest = 0
    end_index_list1 = 0

    # Fill the table based on common elements in both lists
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if list1[i - 1] == list2[j - 1]:
                table[i][j] = table[i - 1][j - 1] + 1
                if table[i][j] > longest:
                    longest = table[i][j]
                    end_index_list1 = i

    # The longest run ends at end_index_list1 in list1
    # Extract the subsequence
    longest_run = list1[end_index_list1 - longest:end_index_list1]
    
    return longest_run

In [30]:
# Now we should have two metrics for how similar they are. 
# You should have a count of how long the greatest common subsequence is between the two lists.  

for i in range(0, 3):
    # Now we will get the longest run between the two lists

    print(f'row {i}')   
     
    sorted_3 = sorted(top3_percent_dict[i].items(), key=lambda x: x[1], reverse=True)
    sorted_5 = sorted(top5_percent_dict[i].items(), key=lambda x: x[1], reverse=True)

    keys_in_order_3 = [x[0] for x in sorted_3]
    keys_in_order_5 = [x[0] for x in sorted_5]

    longest_run_realised = longest_run(keys_in_order_3, keys_in_order_5)

    print('Relative percent of the longest agreed consecutive run', len(longest_run_realised)/max(len(top3_percent_dict[i]), len(top3_percent_dict[i])))

    # print('top3')
    # #pprint(sorted(top3_percent_dict[i].items(), key=lambda x: x[1], reverse=True))
    
    # print('top5')
    # #pprint(sorted(top5_percent_dict[i].items(), key=lambda x: x[1], reverse=True))

    # Calculate the average difference between corresponding keys in the two dicts
    average_difference = sum(abs(top3_percent_dict[i][key] - top5_percent_dict[i].get(key, 0)) for key in keys_in_order_3) / len(keys_in_order_3)
    print("Average difference:", average_difference)

row 0
Relative percent of the longest agreed consecutive run 0.38461538461538464
Average difference: 0.02035963669066504
row 1
Relative percent of the longest agreed consecutive run 0.13333333333333333
Average difference: 0.020784918797258453
row 2
Relative percent of the longest agreed consecutive run 0.21428571428571427
Average difference: 0.014969947934679992


These are not the same. 

- London -> Not the same. 
- Birmingham -> No where near. The smaller ranks are a lot more varied. 
- Newcastle ->